<a href="https://colab.research.google.com/github/JoKangHyeon/Inhatc-2022-Bigdata-YoutubeTrandWordcloud/blob/main/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#필요 라이브러리 설치

!apt-get update -qq
!apt-get install fonts-nanum* -qq 
#!unzip /content/file.zip -d /content
!pip install KoNLPy
!pip install selenium
!pip install bs4
!apt install chromium-chromedriver
!pip install pyblaze
!git clone https://github.com/facebookresearch/fastText.git
!pip install fastText/

import time
import pickle
import json
import re
from time import mktime
from datetime import datetime
from collections import Counter
from os import listdir
from os.path import isfile, join

from selenium import webdriver as wb
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup as bs4
import pandas as pd
from konlpy.tag import Okt
import pyblaze.multiprocessing as xmp

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib import rc
from wordcloud import WordCloud

In [4]:
#playboard 가져오기
url = 'https://playboard.co/youtube-ranking/most-superchatted-all-channels-in-south-korea-total'

chrome_options = wb.chrome.options.Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = wb.Chrome('chromedriver', options=chrome_options)

driver.get(url)

body = driver.find_element(By.CSS_SELECTOR, 'body')
for i in range(1,6):
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    #body.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.5)


soup = bs4(driver.page_source)
title =soup.select('.chart__row')
title

frompb = []

#정치 체널 제외
black_list=['UCzQJmmpZjqzJe96CwlrwlHQ','UCgZrbHKVOCdL4R_ibLGasCQ','UC4y2Jx26qCb7CrSt_i5bf1A','UCn9mJ4htO64-1osMWYu9k5Q','UCrZpcsm1OVzHRJswMxJ2lzg','UC5ORAsVQEmnyOB5isZBT4tA','UCPKAW8V7W4c7TRyZCDQUIrQ','UC40FfFzNGSs7Z1aGZgUHJng',
            'UCXvQXTcC77Nav46NYc4FtDA', '1ENoajw23ec9ItPw0WcA', 'UCLXQlPDLHcTOYWVtj5N6fOg','UCOJMPxYvOS0Zud-gWMNw6eA','UCWziMNlK21nuVw4Rvkveb1A','UCuCECsh_wb-C4loI4xboWVw','UCjBniflu825j2zTJK_g_X7g','UC9Jw0zmPSVf_PpTuUWSb88g',
            'UCdp4_yTBhQmB8E339Lafzow','UCmAtYbRi5KHShhQqmCqo6ng','UCkcviF4GHBAVyaYOkDmyPnw','UCI788NxeJn_gr5GlzmnD3vw','UCmd961UeQtKwsw6_assnw3g', 'UC0M-_02RJqMlGTKUjF1WhJg', 'UCs18sMwqTCMBmTqX0b2-w6A', 'UC1qldNOqaqIY2PwhbVwIhWA',
            'UCgOLQwRv1r2m9mhE1tfsn3Q','UCek6BDHDbP2C5pzbXDAoc4A','UCOqCunaF9qVN8bXwsK0HT3g', 'UCxvU6bRtYhNLvZleAIGa-FQ', 'UCu1FzjrHosuKGvgIx8oBi8w', 'UCY8i60Jd_DPyqOB_AJut2vw', 'UCX4LFdCjFXMaCPbRH1omP6Q', 'UCM8BcGB6BWKq3utIMhGKnUA',
            'UCUxTPRSns--l5BX2537u7Rw', 'UCqqOJp6scEt_sAlIi-Fp8Ow', 'UClW6Qbo0AKEukR36IaLr5Wg','UCFoK0ffj51FnsYeqseWC_sw', 'UC2vZ-y5LAIaqJt5aIp9o-Tg', 'UCJS9VvReVkplPwCIbxnbsjQ','UCfI3DOCx3bgxnZbwOrxu7DA','UCX7-K_PSdtAiUDLEMQwrRoQ',
            'UCv923HGDWMBrnHpC-dkQ3hQ','UCljnbFCt-4doBr7wtEIIbbw','UCkI2g2FvX-haxXqqDMqc96A','UCBghEKEqODLq1qQuN7X9aTA','UCZXrxwYjXWrzE-CZ5XMRNlQ','UCvm91pJsxBES2xBpvVPyeyg','UCj2gpUrZ7LMp4Mp1pVMKz0g','UCSdNAaZWxb4uQmqOpCAsJSw',
            'UCt3XMZlmRh44OYg8RgNzfLg','UC-HXN9uJjNoEQP6NuSsvmyA','UCn0tHAq-Rab_lO1nzcvshKw','UCNRDGWEilNZOIcVIq-FRKZQ','UCDfXwP2JXOTTMaCeUwsuUEA','UCIMv9bOOGWGIfg6wPcRLItQ','UCWpW_GP31BEzlfLaHMD_fPQ','UCMqadVMd0fFKchXVREIDWSA',
            'UCsoWMYTEW2qx-MczuSBJKxg','UCksyDD1Gv6wmpyXJezZkVXw', 'UCzJ76XXo-8n_h7OP1_rDu7w','UCu-1ENoajw23ec9ItPw0WcA','UCLh0F52tTkSkCtViqWt91dg','UCFT-PF577T6OfHobMu5MASw','UCtmvuCbpw6_TFbeO4ToTAIw','UCZG8Bwh4tsaCMaxKifo4lQg',
            'UClAfLVQYZSLrMAQQ_SXPVZw','UCNEmevmeQbD11lV5O-7Zefw','UCSxVWZYk08dNuPcJF33G5eA','UCsgbn3nQIbc0SIohzR64uGw','UCjwnrb7cwSnTBmIxK9EpLyQ','UC8rMw0Si9hXsyB0HZ62Kgiw','UC7GlYUxIZj9zrGZ4qaZdXOA','UCNzh9pp0MiBG7ig6qKvL11w',
            'UCsoiJa5XIqVR2XVzQPITLkA','UCQlm9fWbv3MkmfQLGqwgKgg','UC8_oH_uvZdSGo6lTEUIZOaA','UCxYJ-EJUDJ22yszzRt0mcJA' 'UCMNGwNS4yCEsCCyu2soQDrQ', 'UCQVtV8iONG_gsQIIiP871Jw', 'UCVnsM7nWjRjWIQyBHPFRGDw', 'UC_Vri4o2R5zDXGiTIrnKAaQ',
            'UCZcKfUzxlBXNyAX_mVRTTlQ', 'UCTiPWxlnTbkavHmXFkzldHg', 'UCS7oapzKYfu85wuEtWWHk8A', 'UCh8kNH1ACKADSjesimlUp4A', 'UCGsLaVFoPgTuJUDYW1ygcAg', 'UCmtqPgnS5DiI1xPygbyJ4ww']

for line in title:
  name = line.select('.name > a')
  if(len(name)!=0):
    target= (name[0]['title'][:-33],name[0]['href'][12:])
    if target[1] not in black_list:
      frompb.append(target)

#BS 가공공
def getVideos(link):
  try:
    driver = wb.Chrome('chromedriver', options=chrome_options)
    driver.get('http://www.youtube.com/channel/'+link[1]+'/videos')
    i = 0
    while True:
        scroll_height = 2000
        document_height_before = driver.execute_script("return document.documentElement.scrollHeight")
        driver.execute_script(f"window.scrollTo(0, {document_height_before + scroll_height});")
        time.sleep(1.5)
        document_height_after = driver.execute_script("return document.documentElement.scrollHeight")
        if document_height_after == document_height_before:
            break
        i+=1
        if i>50:
          break
    print(link[0]+" Worker Finished")
    return (link[0],driver.page_source)
  except Exception as e:
    print(e)
    #Pyblaze에 return 안 해주면 프로그램이 멈춤춤
    return ("Failed","Failed")

def parallel(texts):
    tokenizer = xmp.Vectorizer(getVideos, num_workers=10)
    return tokenizer.process(texts)

data = parallel(frompb)

#실패한 항목 제거
while  ("Failed","Failed") in data:
  data.remove( ("Failed","Failed"))

data_souped = {}

#BS 적용용
for d in data:
  soup = bs4(d[1])
  data_souped[d[0]]=soup.select("#video-title-link")

#유튜브 세부 데이터 크롤링
def getDateView(d):
  yt_video_data = []
  for v in d[1][:-1]:
    #제목
    title = v.select("yt-formatted-string")[0].contents[0]

    #View 분리
    splited= v.select("yt-formatted-string")[0]['aria-label'].split(' ')
    view = splited[len(splited)-2]
    view=int(view.replace(',',''))
    try:
      #방송일자 받아오기
      driver = wb.Chrome('chromedriver', options=chrome_options)
      driver.get("https://www.youtube.com"+v['href'])
      #myElem = .until(EC.presence_of_element_located((By.ID, 'factoid style-scope ytd-factoid-renderer')))
      time.sleep(5)
      soup2 = bs4(driver.page_source)
      date= soup2.findAll("div",{"class":"factoid style-scope ytd-factoid-renderer", "role":"text"})[2]['aria-label']  
      date = time.strptime(date,"%b %d, %Y")
      #2021이후 데이터만 정리
      dt = datetime.fromtimestamp(mktime(date))
      if datetime(2021,1,1)>dt:
        break
      else:
        yt_video_data.append({"title":title,"view":view,"date":dt})
    except Exception as e:
      print("https://www.youtube.com"+v['href'])
      print(e)
    finally:
      driver.quit()
  print(d[0]+"date get Finished")
  print(yt_video_data)
  return (d[0],yt_video_data)
  

def parallel2(texts):
    tokenizer = xmp.Vectorizer(getDateView, num_workers=10)
    return tokenizer.process(texts)

p=[]
for d in data_souped:
  p.append((d,data_souped[d]))
#data2 = parallel2(p)

import resource
import sys
#pickle의 max recursion 제한에 걸리는 것을 해결하기 위해 제한 해제제
max_rec = 0x100000
resource.setrlimit(resource.RLIMIT_STACK, [0x100 * max_rec, resource.RLIM_INFINITY])
sys.setrecursionlimit(max_rec)

#중간에 멈출 것을 대비하여 pickle 실행
dateView = None

for d in p:
  dateView = getDateView(d)  
  with open(dateView[0]+'.pickle', 'wb') as f:
    pickle.dump(dateView[1], f, pickle.HIGHEST_PROTOCOL)

nlp = Okt()
counterDict = {}

pickles = [f for f in listdir("/content/content") if isfile(join("/content/content", f)) and f.endswith(".pickle")]

i=0
for p in pickles:
  i+=1
  print(str(i) + " / " + str(len(pickles)))
  with open('/content/content/'+p,'rb') as f:
    dateView = pickle.load(f)
  #print(dateView)
  for d in dateView:
    if 'title' in d.keys():
      d['title'] = re.sub(r'[^\w]', ' ', d['title'])+''
      d['noun'] = nlp.nouns(d['title'])
      #print(d['noun'])
      count = Counter(d['noun'])
      if not d['date'].month in counterDict:
        counterDict[d['date'].month] = {}
      
      #print(count)
      for c in count:
        if not c in counterDict[d['date'].month]:
          counterDict[d['date'].month][c]=0
        counterDict[d['date'].month][c] += count[c]*d['view']

#결과 학인
print(counterDict)  

#1차 시도
#font_path = "/usr/share/fonts/truetype/nanum/NanumGothicEco.ttf"
#font_name = fm.FontProperties(fname = font_path).get_name()
#matplotlib.rc('font', family=font_name)
#fm._rebuild()
#for m in counterDict:
#  wc = WordCloud(font_path, background_color='ivory', width=800, height=600)
#  cloud=wc.generate_from_frequencies(counterDict[m])
#  cloud.to_file(str(m) + '_cloud.jpg')

#매 달마다 중복되는 항목을 제거하여 워드클라우드 생성
font_path = "/usr/share/fonts/truetype/nanum/NanumGothic.ttf"
font_name = fm.FontProperties(fname = font_path).get_name()
matplotlib.rc('font', family=font_name)
fm._rebuild()
for i in range(2,13):
  print(counterDict[i-1])
  for k in counterDict[i-1]:
    if k in counterDict[i]:
      counterDict[i][k]=0
for m in counterDict:
  wc = WordCloud(font_path, background_color='ivory', width=800, height=600)
  cloud=wc.generate_from_frequencies(counterDict[m])
  cloud.to_file(str(m) + '_cloud.jpg')